In [175]:
# unzip
# unzip <src of zip> -d <where to unip>

# check num files
# ls <dir> -1 | wc -l

In [172]:
import os
import math
import shutil
import random
from pathlib import Path


data_path = Path("../data/dataset")
excl_path = ["train", "test"]

# create test and train dirs
for p in excl_path:
    pth = data_path / p
    if not pth.is_dir():
        os.makedirs(pth)

for path in data_path.iterdir():
    if path.is_dir() and path.stem not in excl_path:
        all_images = list(path.glob("*.jpg"))
        test_images = random.sample(population=all_images, k=math.floor(len(all_images)*0.2)) # 20% test
        test_subdir = data_path / "test" / path.stem
        train_subdir = data_path / "train" / path.stem

        # move test images to each respective class (/test/class_name)
        if not test_subdir.is_dir():
            # create subdir for each class
            os.makedirs(test_subdir)

            for img_path_str in test_images:
                img_path = Path(img_path_str)
                shutil.move(img_path_str.absolute(), test_subdir.absolute())
                
        # move the remaining images to train directory
        if not train_subdir.is_dir():
            shutil.move(path, train_subdir)


In [173]:
test = data_path / "test"
for d in test.iterdir():
    print(f"test_{d.stem} => {len(list(d.glob('*.jpg')))}")

test_dew => 139
test_fogsmog => 170
test_frost => 95
test_glaze => 127
test_hail => 118
test_lightning => 75
test_rain => 105
test_rainbow => 46
test_rime => 232
test_sandstorm => 138
test_snow => 124


In [174]:
train = data_path / "train"
for d in train.iterdir():
    print(f"train_{d.stem} => {len(list(d.glob('*.jpg')))}")

train_dew => 559
train_fogsmog => 681
train_frost => 380
train_glaze => 512
train_hail => 473
train_lightning => 302
train_rain => 421
train_rainbow => 186
train_rime => 928
train_sandstorm => 554
train_snow => 497
